# Priprava podatkov

In [84]:
import numpy as np
import pandas as pd

Preberemo datoteko

In [85]:
xls = pd.ExcelFile(r'IJS_database_anonimized.xlsx')
sheet1 = xls.parse(0)
sheet2 = xls.parse(1)
sheet3 = xls.parse(2)

frames = [sheet1, sheet2, sheet3]
df = pd.concat(frames)

Izračunamo starost

In [86]:
from datetime import datetime
#tu bomo uredili stolpce relevantne za starost pri posegu in slednje izračunali

df.rename(columns = {'Datum rojstva':'Datum_rojstva'}, inplace = True)  
df.rename(columns = {'Datum posega':'Datum_posega'}, inplace = True) 
df.at[0,'Datum_rojstva'] = "29.4.1948"
df['Datum_rojstva'] = df['Datum_rojstva'].str.replace(" ", "")
df['Datum_posega'] = df['Datum_posega'].str.replace(" ", "")



def calculate_age(datum_rojstva, datum_posega):
    datum_rojstva = datetime.strptime(datum_rojstva, "%d.%m.%Y").date()
    datum_posega = datetime.strptime(datum_posega, "%d.%m.%Y").date()
    return datum_posega.year - datum_rojstva.year - ((datum_posega.month, datum_posega.day) < (datum_rojstva.month, datum_rojstva.day))


df['starost'] = df.apply(lambda x: calculate_age(x.Datum_rojstva, x.Datum_posega), axis=1)

df.drop('Datum_posega', axis=1, inplace=True)
df.drop('Datum_rojstva', axis=1, inplace=True)

Stolpce, kjer manjka več kot petina podatkov izbrišemo

In [87]:
manjkajoce = df.isnull().sum()

meja = 0.2
counter = 0 #steje koliko stolpcev smo pometali ven
for col in df.columns:
    if manjkajoce[col] > 0:
        if manjkajoce[col] < len(df)*meja:
            pass
        else:
            print(col, round(manjkajoce[col]/len(df[col]),2))
            df.drop(col, axis=1, inplace=True)
            counter += 1
    
#print(df.isnull().sum())

Tarčna stena 0.3
IVS 0.29
ALT 0.78
bilirubin (dir) 0.88
Način TX 0.76
urat 0.47
LDL 0.2
Fe 0.47
EDV 0.35
ESV 0.37
L 0.24
Hb 0.24
feritin 0.42
TRC 0.24
RDW 0.24
Komentar kontrole 0.24


Dodamo še tip ultrazvoka (cifra_excela)

In [88]:
#dodajmo tabeli še stolpec iz katerega lista (excel sheet) je bil podatek prvotno
df.columns = ['cifra_pacienta',"spol","glavna_diagnoza","responder","ef","lvedd","vti","Na","K","secnina","kreatinin","ast","AF","gGT","bilirubin_cel","pro_BNP","CRP","razdalja","holesterol","HDL","trigliceridi","glukoza","starost"]
cifra_excela = pd.Series([])
df = df.reset_index()

for i in range(len(df)):
    if df["cifra_pacienta"][i] in range(1,30):
        cifra_excela[i]=1
 
    elif df["cifra_pacienta"][i] in range(30,58):
        cifra_excela[i]=2
    
    else:
        cifra_excela[i]=3

df.insert(2, "cifra_excela", cifra_excela)

C:\Users\borbr\AppData\Local\Temp\ipykernel_5684\4232662959.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cifra_excela = pd.Series([])


In [89]:
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
df.drop(df.filter(regex="cifra_pacienta"),axis=1, inplace=True) # znebimo se še unnamed index stolpca
df.drop(df.filter(regex="index"),axis=1, inplace=True) # indeks pride od drugacnih listov (vsak list posebej zacne stevilciti) in ni pomemben

Kategoricne damo v 0 1

In [90]:
df_one = pd.get_dummies(df["spol"])
df_two = pd.concat((df_one, df), axis=1)
df_two = df_two.drop(["spol"], axis=1)
df_two = df_two.drop(["M"], axis=1)
df = df_two.rename(columns={"Ž": "spol"})

df_one = pd.get_dummies(df["glavna_diagnoza"])
df_two = pd.concat((df_one, df), axis=1)
df_two = df_two.drop(["glavna_diagnoza"], axis=1)
df_two = df_two.drop(["Neishemična KMP"], axis=1)
df = df_two.rename(columns={"Ishemična KMP": "glavna_diagnoza"})

Koliko vrstic smo pometali ven (ce manjka vec kot petina podatkov)

In [91]:
vse_vrstice = df.shape[0]
df = df[df.isnull().sum(axis=1) < 0.2 * len(df.columns)]
nove_vrstice = df.shape[0]
print(round(((vse_vrstice - nove_vrstice) / vse_vrstice),2))

#Koliko stolpcev smo pometali ven
print(round(((counter) / (len(df.columns)+counter)),2))

0.07
0.41


In [92]:
df.to_excel("data.xlsx")